| Version | Published Date| Details |
| -- | -- | -- |
| ver.1.0.0 | 2023/8/29 | 初版 |

# Google スプレッドシートで自動化を学ぼう

Googleスプレッドシート (Google Sheets) は無料で使えるWebブラウザベースの表計算アプリケーションです。Googleアカウントを持っていればだれでも利用可能であり，便利で機能が豊富であることから Microsoft Office Excel の競合製品になっています。

Google Sheets には独自のAPI (APIという単語を知らなければ調べてみましょう) がありますが，それをすべて学んで使うのはやや面倒です。今回は `gspread` というサードパーティーモジュールを使って Google Colab から Google Sheets を利用します。

Pythonを使ってスプレッドシートを操作することで、さまざまな情報をスプレッドシートに集約したり、より柔軟な計算処理をおこなうことができます。

まずはPythonのパッケージ管理ツールである `pip` を使って `gspread` をアップデートしておきます。

In [1]:
!pip install --upgrade gspread

パッケージのインストールが完了したら、このColaboratoryの**ランタイムを再起動**しましょう。

ページ上部のメニューから [ランタイム] -> [ランタイムを再起動] を実行してください。
再起動していない場合、インストールしたパッケージが`import`で正常に読み込まれない場合があります。

うまく`import`できない場合はランタイムの再起動を試してみるようにしてください。

## Google スプレッドシートを認証する

`gspread` を使う前に，まずは `gspread` を認証しましょう。「認証」は、ユーザーが正当なユーザーであることを確認するためのプロセスです。ユーザー名とパスワード、二要素認証、公開鍵暗号などの方法を使って、システムへのアクセスを制限し、セキュリティを保護します。

以下のセルを実行すると，自動的にポップアップが開きます。

1. Google認証情報へのアクセスの許可
2. アクセスを許可するアカウント **(もし複数アカウントでGoogle Colabを使用している場合は注意が必要です)**
3. Colabolatory Runtimes に対する Google アカウントへのアクセスを許可

の手順でそれぞれ許可しましょう。

In [2]:
from google.colab import auth
from google.auth import default
import gspread

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

## スプレッドシートの作成，アップロード，一覧

新規の `Spreadsheet` オブジェクトは，既存のスプレッドシートや，空白のスプレッドシート，アップロードされたスプレッドシートから作成できます。既存のGoogle SheetsのスプレッドシートからSpreadsheetオブジェクトを作成するには，スプレッドシートのIDが必要です。スプレッドシートのIDは，URLの `spreadsheets/d/` と `/edit` の間の部分です。

たとえば https://drive.google.com/file/d/13maFkPcxHrKm5Pa2pAszmNU_m3-BnpVn/view?usp=sharing のURLのIDは `13maFkPcxHrKm5Pa2pAszmNU_m3-BnpVn` です。

In [ ]:
workbook = gc.open_by_key('13maFkPcxHrKm5Pa2pAszmNU_m3-BnpVn')
workbook.title

またIDではなくURLを使うことでもスプレッドシートにアクセスすることができます。それ以外にもファイル名を使った `gc.open()` でもスプレッドシートを開くことができます。

In [ ]:
workbook = gc.open_by_url('ss_url')
workbook.id

### 空白のスプレッドシートを作成

新規に空白のスプレッドシートを作成するには `gc.create()` を使用します。

In [ ]:
new_workbook = gc.create('新しいスプレッドシート')

ColabにログインしているGoogleアカウントのGoogle Driveを見ると「新しいスプレッドシート」という名のスプレッドシートが追加されているはずです。

## スプレッドシートの削除

スプレッドシート自体の削除には `del_spreadsheet()` を使います。ゴミ箱に入らず，完全に削除されることに注意してください。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
gc.del_spreadsheet(new_workbook.id)

自身のGoogleアカウントに紐づくスプレッドシートの一覧は `gc.openall()` を使います。 スプレッドシートのタイトルとidの組み合わせが返ってきます。

In [ ]:
gc.openall()[:10]

## スプレッドシートの属性

ここからの操作は上記のスプレッドシートではなく，自身のドライブにコピーして作成したスプレッドシートを用いて作業をしてください。以下の手順に従いましょう。

1.  [サンプルのデータセットが登録されているスプレッドシート](https://drive.google.com/file/d/13maFkPcxHrKm5Pa2pAszmNU_m3-BnpVn/view?usp=sharing) を開く。
2. ファイル→コピーを作成 を選択
3. わかりやすい場所にコピーを作成する
4. コピーが完了したら，自身のドライブに作成したスプレッドシートのURLをコピーする

In [ ]:
url = 'ss_url' # ↑でコピーしたスプレッドシートのURLを入力
workbook = gc.open_by_url(url)

スプレッドシートのURLを直接参照することもできます。

In [ ]:
workbook.url    # スプレッドシートのURL

スプレッドシートが持つシートにアクセスするには `worksheets()` メソッドを利用します。

In [ ]:
workbook.worksheets()   # シートオブジェクトの一覧

In [ ]:
workbook.worksheets()[0]

`worksheet()` メソッドにシートのタイトルを渡すことでもアクセス可能です。

In [ ]:
workbook.worksheet('titanic')

`add_worksheet()` メソッドを使うと新たにシートを追加できます。

In [ ]:
worksheet = workbook.add_worksheet(title="newsheet", rows=100, cols=20)
workbook.worksheets()

シートを削除するには `del_worksheet()` メソッドを使います。 `newsheet` が削除されていることを確認してください。

In [ ]:
workbook.del_worksheet(worksheet)
workbook.worksheets()

オンラインでだれかがスプレッドシートを変更したら `fetch()` メソッドを呼び出します。これにより `Spreadsheet` オブジェクトを更新してオンラインデータを反映できます。

## スプレッドシートをダウンロードする

スプレッドシートをダウンロードする前に，Colab上では少し工夫が必要です。Colabの環境は通常のPythonの実行環境とは異なるからです。 `gspread` やその他のライブラリにとって，実行している環境が通常のPython実行環境なのか，Colab上なのかはわかりません。

そのため今回はGoogle DriveをColab上のPython実行環境にマウントし，通常のファイルシステムとして扱うための準備をします。「Googleスプレッドシートを認証する」で行った手順と同様に，ColabからGoogle Driveを扱うための許可をしてください。

In [ ]:
from google.colab import drive
root_path = '/content/drive'
drive.mount(root_path)

もうひとつの準備として，ファイルを保存するためのパスを指定します。ここでは `os.path.join()` メソッドを用いています。Pythonの文字列連結を使って

```python
'root_path' + '/' + 'My Drive'
```

のようにしてもよいのですが `/` を入れ忘れたり余分につけてしまったりとバグを生むことがあります。そのためPythonでパスを編集するには `os.path.join()` を使うようにしましょう。

In [ ]:
import os
filename = 'dataset.csv'
my_drive_path = os.path.join(root_path, 'My Drive')
my_file_path = os.path.join(my_drive_path, filename)

続いて `gspread.utils` から `ExportFormat` を `import` し，`csv_data` をバイト列として書き出します。それを `with` 句を使って `my_file_path` に保存します。

自身のGoogle Driveを開いてみましょう。 `dataset.csv` という名前のCSVファイルが保存されていることを確認しましょう。今回はCSV形式でファイルを保存していますが，CSVは複数シートを保存できません。先頭のシートのみ保存されることに注意してください。CSV以外の対応ファイル形式については [公式ドキュメント](https://docs.gspread.org/en/latest/api/utils.html#gspread.utils.ExportFormat) を参照してください。

In [ ]:
from gspread.utils import ExportFormat

csv_data = workbook.export(format=ExportFormat.CSV)
with open(my_file_path, 'wb') as f:
    f.write(csv_data)

## データを読み書きする

Excelと同様に，Googleスプレッドシートのシートも値を持ったセルが行と列に並んでいます。 `[]` を用いてセルを読み書きできます。新規にスプレッドシートをつくり，データを追加してみましょう。

In [ ]:
workbook = gc.create('新しいスプレッドシート')
sheet = workbook.worksheets()[0]    # 最初のスプレッドシートを取得

In [ ]:
sheet.title

セルに値を設定するには `update_acell()` メソッドを使います。

In [ ]:
sheet.update_acell('A1', 'Name')

In [ ]:
sheet.update_acell('B1', 'Age')

In [ ]:
sheet.update_acell('C1', 'Favourite Movie')

セルの値を取得するには `acell()` メソッドを使ってセルの座標を指定したあと，`value` を使います。

In [ ]:
sheet.acell('A1').value

空のセルにアクセスすると `None` が返ります。

In [ ]:
sheet.acell('A2').value is None

`cell()` メソッドを用いてx座標とy座標の形式でセルを指定することもできます。Pythonのリストと違いGoogleスプレッドシートの世界では，添字が0ではなく1から始まることに注意してください。

座標 `(1, 2)` は `B1` と同じ場所を指します。

In [ ]:
sheet.cell(1, 2).value

In [ ]:
sheet.update_acell('A2', 'Matt')
sheet.update_acell('B2', 30)
sheet.update_acell('C2', 'Titanic')

Name | Age | Favourite Movie
-- | -- | --
Matt | 30 | Titanic

シートを開いて，このようなシートが作成されていることを確認しましょう。

## 列と行のアドレス指定

Googleスプレッドシートのセルのアドレス指定方法はExcelと同じです。Pythonのインデックスは `0` から始まりますが，Googleスプレッドシートの列と行のインデックスは `1` から始まることに注意しましょう。つまり最初の行や列のインデックスは `0` ではなく `1` です。

`a1_to_rowcol` や `rowcol_to_a1` を使うと文字列形式のアドレスと `(列, 行)` のタプル形式のアドレスを相互変換できます。

In [ ]:
from gspread.utils import a1_to_rowcol, rowcol_to_a1

a1_to_rowcol('A2')

In [ ]:
rowcol_to_a1(2, 1)

## 列や行全体を読み書きする

`Spreadsheet` オブジェクトが最初に読み込まれたとき `Worksheet` オブジェクトのすべてのデータが読み見込まれているため，セルの値を読み出すのは瞬時です。一方でセルの値をスプレッドシートに書き込むのにはネットワーク接続が必要なので，1秒程度の時間がかかります。たくさんのセルを一気に更新すると，かなり時間がかかります。

そのため `Worksheet` には列や行全体のデータを一度に読み書きするメソッドがあります。 `col_values()` `row_values()` はそれぞれ列と行全体を読み込み `update()` メソッドに範囲を指定することで一括で値を更新できます。

まず準備として **スプレッドシートの属性** の章でサンプルのデータセットが登録されたスプレッドシートからコピーしてきたシートのURLを入力しましょう。

In [ ]:
url = 'ss_url' # コピーしたスプレッドシートのURLを入力
workbook = gc.open_by_url(url)
titanic = workbook.worksheet('titanic')

In [ ]:
col_names = titanic.row_values(1)
col_names

In [ ]:
titanic.col_values(1)

In [ ]:
titanic.row_values(3)

列を一気に更新したいときは列の範囲を指定し，新しい列のリストを渡します。このとき与えるのは `new_row` ではなく `[new_row]` つまり「リストのリスト」になっていることに注意します。

In [ ]:
new_row = [
 '2',
 '0',
 '0',
 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)',
 'female',
 '38',
 '1',
 '0',
 'PC 17599',
 '71.2833',
 'C85',
 'C']
titanic.update('A3:L3', [new_row])

同様に列を一括で更新しましょう。今回は `col_names.index('Name')` で `Name` が入っているカラムの添字を取り出します。スプレッドシートの添字は1から始まるので `+1` し，乗客の名前すべてを取り出しましょう。

In [ ]:
names = titanic.col_values(col_names.index('Name') + 1)

`upper()` メソッドを使ってそれぞれの名前を大文字にし `capital_names` に格納します。列を一括で更新する場合，リストのリスト，つまり入れ子のリストになることに注意します。

In [ ]:
capital_names = []
for name in names:
    capital_name = name.upper()
    capital_names.append([capital_name])

In [ ]:
titanic.update('D2', capital_names)

## Pandasを使った読み書き

しかしながら少し高度な集計や更新をしようとすると，Pandasを使いたくなるでしょう。 `gspread` にはPandasとの連携機能もあります。`get_all_records()` メソッドを使うと直接 `DataFrame` を作成できます。

In [ ]:
import pandas as pd
df = pd.DataFrame(titanic.get_all_records())
df

運賃を2倍してみます。

In [ ]:
df["Fare"] = df["Fare"] * 2

更新した `DataFrame` をアップロードするには以下のようなコードを使います。このコードはシート全体を `DataFrame` で置き換えます。毎回の作業ごとにアップロードするのではなく `DataFrame` 上での作業が終了したらアップロードするようにしましょう。

In [ ]:
titanic.update([df.columns.values.tolist()] + df.values.tolist())

# 確認テスト

1から1,000までの数が書かれたくじを考えます。ここでは1回引いたくじは箱の中に戻します。Pythonでこのくじについての実験を行うことを考えます。 `random` モジュールの `random.choice()` を使うとあるリストから要素をひとつ取り出してくることができます。

## (1) 10,000回くじを引く

Pythonの `random` はほんとうにランダムでしょうか。これを確かめるために，以下の手順に従ってプログラムを完成させ，問いに答えてください。

1. くじから数字を10,000回取り出し，結果をリストに保存する
1. 結果をPandasの `DataFrame` に変換する
1. [1から1000までの数のシート](https://drive.google.com/file/d/1NOsZ8D2OStT3yoZixO7OBmRgTd9HoDAb/view?usp=sharing) を開き，自分のGoogleスプレッドシートにコピーする
2. コピーしたスプレッドシートを開き `random_result` という名前のシートを追加する
3. 2で作った `DataFrame` をアップロードする

上記のコードが完成したらプログラムを実行しましょう。実行後、当該のスプレッドシートを開いて以下の手順を続けてください。

1. 「結果」シートの `B2` セルに `=countif(random_result!B2:B10001, A2)` をコピーして貼り付ける
1. セルの右下にカーソルを動かし，カーソルが `+` に変わるのを確認する
1. 確認できたらダブルクリックし，式が `B` 列の最下まで入力されるのを確認する
1. グラフが描画されていることを確認する
1. くじの数字の出現回数の最大値をグラフから読み取る

確認テストの回答欄には **「くじの数字の出現回数の最大値」を** 数字で入力してください。

> 【こみみ情報】
>
> 数学的な性質を知っていればこの答えはすぐに導き出すことができます

In [ ]:
import random
import pandas as pd
random.seed(100)

results = []
lottery = list(range(1, 1000 + 1)) # 1から1000までの数が含まれているくじ
for i in range(1, 10000 + 1):
    selected = random.choice(lottery)
    result = (i, selected)
    results.append(result)

url = 'ss_url'
workbook = gc.open_by_url(url)
workbook.add_worksheet(title="random_result",rows=10000,cols=2)  # ワークシートを追加する。rowsとcolsを正しく指定しましょう。
df = pd.DataFrame(results, columns=["times", "number"])
# 追加したワークシートを df で置き換える

In [ ]:
print(df)

In [ ]:
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

## (2) 1000回引けばあたりは出るか

Pythonの `random` がランダムであることはわかりましたか。それでは実際にくじをひきましょう。ここでは `777` があたりのくじとします。今回の例で，このくじを1000回引いたときに、**「あたりを引く確率」**を答えてください。以下のサンプルコードを使いましょう。回答は四捨五入し、0~100の間の整数で入力してください。単位は "%" です。

In [ ]:
import random
random.seed(100)

length = 1000
lottery = list(range(1, length + 1))  # 1から1000までの数が含まれている
results = []
times = 1000
lucky_number = 777

for i in range(times):
    for j in range(length):
        selected = random.choice(lottery)# lotteryから数字を取り出してきて selected に格納する
        if selected == lucky_number:
            # results に値を格納する
           result = (i, selected)
           results.append(result)
           break
    else:
         print(i, "はずれでした")

In [ ]:
# 確率の計算
# 回答は四捨五入し、0~100の間の整数で入力してください。単位は "%" です。
len(results) / length * 100